In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import gym
import random

from keras import __version__
tf.keras.__version__ = __version__

from gym import Env
from gym.spaces import Discrete, Box, Dict
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory



ImportError: cannot import name '__version__' from 'tensorflow.keras' (c:\Python311\Lib\site-packages\keras\api\_v2\keras\__init__.py)

In [4]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [5]:
episodes = 100
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, truncated,info = env.step(action)
        score += reward
    print("Episode {} Score: {}".format(episode, score))

Episode 1 Score: 19.0
Episode 2 Score: 15.0
Episode 3 Score: 49.0
Episode 4 Score: 28.0
Episode 5 Score: 14.0
Episode 6 Score: 18.0
Episode 7 Score: 19.0
Episode 8 Score: 12.0
Episode 9 Score: 12.0
Episode 10 Score: 16.0
Episode 11 Score: 16.0
Episode 12 Score: 11.0
Episode 13 Score: 10.0
Episode 14 Score: 17.0
Episode 15 Score: 11.0
Episode 16 Score: 18.0
Episode 17 Score: 45.0
Episode 18 Score: 18.0
Episode 19 Score: 9.0
Episode 20 Score: 26.0
Episode 21 Score: 16.0
Episode 22 Score: 12.0
Episode 23 Score: 15.0
Episode 24 Score: 11.0
Episode 25 Score: 10.0
Episode 26 Score: 23.0
Episode 27 Score: 13.0
Episode 28 Score: 17.0
Episode 29 Score: 14.0
Episode 30 Score: 25.0
Episode 31 Score: 9.0
Episode 32 Score: 15.0
Episode 33 Score: 31.0
Episode 34 Score: 52.0
Episode 35 Score: 12.0
Episode 36 Score: 19.0
Episode 37 Score: 17.0
Episode 38 Score: 53.0
Episode 39 Score: 21.0
Episode 40 Score: 23.0
Episode 41 Score: 16.0
Episode 42 Score: 24.0
Episode 43 Score: 14.0
Episode 44 Score: 31.0

c:\Python311\Lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(
c:\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


## Custom environment

In [116]:
# Define the WaterTank environment
class WaterTank(Env):
    def __init__(self, setpoint = 10.0, dt = 1e-3, tf = 10):
        """
        The tank is governed by the following first-order differential equation
        
        A(dH/dt) = bV - a*sqrt(H)

        The states of the system are: H
        The inputs of the system are: v
        The outputs of the system are: H

        H:  Height of water
        V: voltage applied to the pump

        """

        # Possible actions
        self.action_space = Discrete(5) # open, close for output flows, increase/decrease/stay for pump voltage
        
        # Water level temperature array
        #self.observation_space = Box(low = np.array([1e-6]), high = np.array([200]))
        self.observation_space = np.array([
            Box(low = np.array([1e-6]), high = np.array([200])),
            Box(low = np.array([1e-6]), high = np.array([500])),
            Box(low = np.array([1e-6]), high = np.array([100]))
        ])

        # self.observation_space = Dict({
        #     "H": Box(low = np.array([1e-6]), high = np.array([200])),
        #     "Q": Box(low = np.array([1e-6]), high = np.array([500])),
        #     "V": Box(low = np.array([1e-6]), high = np.array([100]))
        # })

        # Tank parameters
        self.H0 = 100 + np.random.uniform(-100, 100) # initial level
        self.A = 20 # cross sectional area of the tank

        # Initialize current water level
        self.H = self.H0 # meters

        # Set desired level
        self.setpoint = setpoint

        # Input valve parameters
        self.b = 5 # Constant related to the flow rate into the tank
        self.V = 5 + np.random.uniform(-5, 10) # Initial voltage of the pump
        self.P = self.V*self.b

        # Output valve parameters
        self.a = 2 # Constant related to the flow rate out of the tank
        self.Q = self.a*math.sqrt(self.H) # Output flow depends of pressure which depends of water height

        self.dt = dt # each iteration is a 10ms time-step
        self.tf = tf

        # Initial water level, inflow and outflow
        self.state = self.H

        self.t = 0


    def step(self, action):
        """
            action 0: Open outflow valve
            action 1: Close outflow valve
            action 2: Increase voltage for pump
            action 3: Decrease voltage for pump
            action 4: Stay in same voltage
        """
        if action == 0: # Open outflow valve
            self.Q = self.a*math.sqrt(self.H)
        elif action == 1: # close outflow valve
            self.Q = 0.0
        elif action == 2: # Increase pump voltage
            self.V += 0.5
        elif action == 3: # Decrease pump voltage
            self.V -= 0.5

        # Update inflow
        self.P = self.b*self.V

        # Output flow
        self.Q = self.a*math.sqrt(self.H)

        # Update tank level using trapezoidal method
        Hold = self.H
        Hnew = Hold + (self.dt/2.0)*((self.P - self.Q)/self.A)
        #self.H += ((self.P - self.Q)/self.A) * self.dt # Trapezoidal method
        self.H = Hnew
        if self.H <= 0:
            self.H = 1e-6

        # Compute reward as the negative abs difference between current level and setpoint
        
        reward = -abs(self.H - self.setpoint)  # Negative absolute difference as reward

        # Update time
        self.t += self.dt

        done = self.t >= self.tf

        info = {}
        #self.state = self.H
        self.state = np.array([self.H, self.Q, self.V])
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        # Initialize current water level
        self.H0 = 100 + np.random.uniform(-100, 100) # initial level
        self.H = self.H0 # meters

        # Input valve parameters
        self.V = 5 + np.random.uniform(-5, 10) # Initial voltage of the pump
        self.P = self.V*self.b

        # Output valve parameters
        self.Q = self.a*math.sqrt(self.H) # Output flow depends of pressure which depends of water height

        reward = -abs(self.H - self.setpoint)  # Negative absolute difference as reward

        #self.state = self.H
        self.state = np.array([self.H, self.Q, self.V])

        self.t = 0

        return self.state



In [117]:
env = WaterTank(
    tf = 5
)

c:\Python311\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [118]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done,info = env.step(action)
        score += reward
    print("Episode {} Score: {}".format(episode, score))

Episode 1 Score: -336893.1790654605
Episode 2 Score: -385184.6690240174
Episode 3 Score: -544584.665061998
Episode 4 Score: -34300.66411421761
Episode 5 Score: -60013.70736119558
Episode 6 Score: -387399.4643059907
Episode 7 Score: -839261.6482947398
Episode 8 Score: -380153.60191190935
Episode 9 Score: -939425.7181825117
Episode 10 Score: -360593.8153474734


## NN Model

In [122]:
def build_model(states, actions):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape = (1, states)))
    model.add(tf.keras.layers.Dense(24, activation = 'relu'))
    model.add(tf.keras.layers.Dense(24, activation = 'relu'))
    model.add(tf.keras.layers.Dense(actions, activation = 'linear'))
    return model

In [123]:
env.observation_space.shape

(3,)

In [124]:
states = env.observation_space.shape[0]
actions = env.action_space.n
print(states, actions)

model = build_model(states, actions)
model.summary()

3 5
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 3)                 0         
                                                                 
 dense_47 (Dense)            (None, 24)                96        


                                                                 
 dense_48 (Dense)            (None, 24)                600       
                                                                 
 dense_49 (Dense)            (None, 5)                 125       
                                                                 
Total params: 821 (3.21 KB)
Trainable params: 821 (3.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [125]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [126]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(
        model = model,
        memory = memory,
        policy = policy,
        nb_actions = actions,
        nb_steps_warmup = 10,
        target_model_update = 1e-2
    )

    return dqn

In [127]:
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate = 1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 34:55 - reward: -131.1485

c:\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\Python311\Lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 59s 6ms/step - reward: -68.0554
2 episodes - episode_reward: -340276.758 [-606901.177, -73652.338] - loss: 1379.688 - mae: 2619.246 - mean_q: -3267.690

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -159.8034
2 episodes - episode_reward: -799017.203 [-802153.233, -795881.173] - loss: 4396.565 - mae: 5500.212 - mean_q: -6865.637

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -111.4144
2 episodes - episode_reward: -557071.849 [-786294.711, -327848.988] - loss: 11792.746 - mae: 8037.651 - mean_q: -10030.938

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -32.3779
2 episodes - episode_reward: -161889.507 [-318136.749, -5642.264] - loss: 12713.874 - mae: 7162.763 - mean_q: -8935.025

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 66s 7ms

In [128]:
scores = dqn.test(env, nb_episodes = 100, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -85890.951, steps: 5000
Episode 2: reward: -470011.374, steps: 5000
Episode 3: reward: -451272.060, steps: 5000
Episode 4: reward: -480266.604, steps: 5000
Episode 5: reward: -481224.517, steps: 5000
Episode 6: reward: -442864.667, steps: 5000
Episode 7: reward: -45584.604, steps: 5000
Episode 8: reward: -490902.595, steps: 5000
Episode 9: reward: -362091.592, steps: 5000
Episode 10: reward: -448551.000, steps: 5000
Episode 11: reward: -320157.492, steps: 5000
Episode 12: reward: -47872.227, steps: 5000
Episode 13: reward: -481396.881, steps: 5000
Episode 14: reward: -509398.773, steps: 5000
Episode 15: reward: -468718.254, steps: 5000
Episode 16: reward: -321360.283, steps: 5000
Episode 17: reward: -302878.631, steps: 5000
Episode 18: reward: -29094.911, steps: 5000
Episode 19: reward: -220312.110, steps: 5000
Episode 20: reward: -77147.905, steps: 5000
Episode 21: reward: -186861.263, steps: 5000
Episode 22: reward: -47195.644, steps: 5